# Fashion MNIST Classifier

In [23]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

# Download and prepare the Fashion MNIST data set 
train_dataset = dsets.FashionMNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.FashionMNIST(root='./data', train=False, transform=transforms.ToTensor())

'\nThe data features can be extracted as train_dataset/test_dataset.data\nThe labels can be extracted as train_dataset/test_dataset.targets \n\n\n'

In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

batch_size = 32

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)


In [48]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride = 1, padding=0)
        self.fc1   = nn.Linear(in_features=6*12*12, out_features=10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.softmax(self.fc1(x),dim=1)
        return x

In [41]:
def train(model, device, train_loader, criterion, optimizer, epoch):
    train_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if batch_idx % (len(train_loader)//2) == 0:
            print('Train({})[{:.0f}%]: Loss: {:.4f}'.format(
                epoch, 100. * batch_idx / len(train_loader), train_loss/(batch_idx+1)))

def test(model, device, test_loader, criterion, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = (test_loss*batch_size)/len(test_loader.dataset)
    print('Test({}): Loss: {:.4f}, Accuracy: {:.4f}%'.format(
        epoch, test_loss, 100. * correct / len(test_loader.dataset)))

In [52]:
num_epochs = 10
model = Model()
model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer =  optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)
scheduler = scheduler = optim.lr_scheduler.MultiStepLR(optimizer,milestones=[5],gamma=0.1)
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, criterion, optimizer, epoch)
    test(model, device, test_loader, criterion, epoch)
    scheduler.step()
    print('Optimizer Learning rate: {0:.4f}'.format(optimizer.param_groups[0]['lr']))

Train(1)[0%]: Loss: 2.3032
Train(1)[50%]: Loss: 1.7359
Train(1)[100%]: Loss: 1.6970
Test(1): Loss: 1.6577, Accuracy: 81.2200%
Optimizer Learning rate: 0.0010
Train(2)[0%]: Loss: 1.5691
Train(2)[50%]: Loss: 1.6401
Train(2)[100%]: Loss: 1.6320
Test(2): Loss: 1.6343, Accuracy: 83.4400%
Optimizer Learning rate: 0.0010
Train(3)[0%]: Loss: 1.5791
Train(3)[50%]: Loss: 1.6180
Train(3)[100%]: Loss: 1.6166
Test(3): Loss: 1.6246, Accuracy: 84.2800%
Optimizer Learning rate: 0.0010
Train(4)[0%]: Loss: 1.6791
Train(4)[50%]: Loss: 1.6090
Train(4)[100%]: Loss: 1.6074
Test(4): Loss: 1.6169, Accuracy: 84.9500%
Optimizer Learning rate: 0.0010
Train(5)[0%]: Loss: 1.6482
Train(5)[50%]: Loss: 1.6039
Train(5)[100%]: Loss: 1.6008
Test(5): Loss: 1.6119, Accuracy: 85.7100%
Optimizer Learning rate: 0.0001
Train(6)[0%]: Loss: 1.5695
Train(6)[50%]: Loss: 1.5906
Train(6)[100%]: Loss: 1.5911
Test(6): Loss: 1.6036, Accuracy: 86.3900%
Optimizer Learning rate: 0.0001
Train(7)[0%]: Loss: 1.6416
Train(7)[50%]: Loss: 1.58